# Cas Kaggle: 

## Introducció

Els telèfons intel·ligents s'han covertit en una eina necessària en el nostre dia a dia ja sigui per treballar, contactar amb els nostres éssers estimats o passar el temps mort revisant les xarxes socials. Tot i així, es requereix d'un element imprescindible que en gran part diferencia el telèfon móbil amb el que ara anomenem telèfon intel·ligent: les aplicacions. 

Actualment existeixen milions d'aplicacions diferents disponibles per a la seva descàrrega i és decisió de l'usuari escollir quina és la que s'adequa a les seves necessitats.

<img src="images/top-apps.jpg" width="40%">

El manteniment dels serveis d'aquestes aplicacions pot ser un treball costós temporal o financerament i les persones encarregades del desenvoluament de noves aplicacions poden necessitar saber de quina forma es rebrà el seu projecte al públic general. 

És per aquesta raó que analitzarem el dataset de Kaggle [Google Play Store Apps](www.kaggle.com/datasets/lava18/google-play-store-apps), on s'emmagatzema informació sobre 10 mil de les aplicacions disponibles a la Play Store, amb l'objectiu de crear un model predictiu amb el qual obtenir la puntuació atribuïda a l'aplicació per part dels usuaris a partir dels altres atributs de l'aplicació.


Per fer-ho seguirem els passos següents:

* Preparació i neteja de les dades

* Anàlisi de les dades

* Selecció del model

* Crossvalidation

* Anàlisi de resultats


## 1. Preparació i neteja de les dades

En aquest primer apartat analitzarem quines són les dades disponibles al dataset i les prepararem i netejarem per el seu ús en l'entrenament i testejat del model d'aprenentatge.

### 1.1. Informació de les dades

Per començar carreguem el dataset des del fitxer i mostrem algunes files.

In [9]:
import numpy as np
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt

# Carreguem el dataset
dataset = pd.read_csv('data/googleplaystore.csv', header=0, delimiter=',')
data = dataset.values

print("Dimensionalitat de la BBDD:", dataset.shape)
dataset.head(5)

Dimensionalitat de la BBDD: (10841, 13)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Observem que el dataset compta amb un total de 13 atributs:

* **App:** Nom de l'aplicació.
* **Category:** Categoria assignada a laplicació.
* **Rating:** Punctuació mitjana dels usuaris. Valor decimal entre 0 i 5.
* **Reviews:** Nombre de ressenyes rebudes dels usuaris.
* **Size:** Tamany de l'aplicació en Bytes.
* **Installs:** Nombre aproximat d'instal·lacions.
* **Type:** Tipus indicant si l'aplicació és de pagament.
* **Price:** Preu de compra.
* **Content Rating:** Indica el públic a qui va dirigida l'aplicació.
* **Genres:** Gèneres assignats a més de la categoría principal.
* **Last Updated:** Data de la darrera actualització emesa.
* **Current Ver:** Versió actual.
* **Android Ver:** Versions d'Android Compatibles.


A continuació comprovem si existeixen elements buits per cadascún dels atributs.

In [10]:
print("Per comptar el nombre de valors no existents:")
print(dataset.isnull().sum())

Per comptar el nombre de valors no existents:
App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64


Observem que als atributs *Rating*, *Type*, *Content Rating*, *Current Ver* i *Android Ver* en tenen algún element inexistent. Per tant caldrà omplir aquests buits.